# Deteksi Stunting

# Deteksi Stunting

## Import Library

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time

from sklearn.preprocessing import label_binarize

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.multioutput import MultiOutputClassifier
from sklearn.preprocessing import OrdinalEncoder
from sklearn import tree

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import roc_curve, auc
from itertools import cycle

import warnings

import pickle

In [ ]:
# Suppress FitFailedWarning
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

### Load Dataframe

In [ ]:
df = pd.read_csv('/home/rmfarizky/Project/deteksi_stunting/Deteksi_Stunting/Classification/data_17-1.csv')
# df = pd.read_csv('/home/rmfarizky/Project/deteksi_stunting/DRAFT/Dataset/new_draft/final_data.csv')


In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
def visualize_data (x):
    fig, axs = plt.subplots(1, 3, figsize=(15, 5))
    x.iloc[:,-3].value_counts().plot.bar(color='coral', ax=axs[0])
    axs[0].set_title('BB/U Distribution')
    axs[0].set_yticks(range(0, max(x.iloc[:, -3].value_counts()) + 1, 100))

    x.iloc[:,-2].value_counts().plot.bar(color='coral', ax=axs[1])
    axs[1].set_title('TB/U Distribution')
    axs[1].set_yticks(range(0, max(x.iloc[:, -2].value_counts()) + 1, 100))

    x.iloc[:,-1].value_counts().plot.bar(color='coral', ax=axs[2])
    axs[2].set_title('BB/TB Distribution')
    axs[2].set_yticks(range(0, max(x.iloc[:, -1].value_counts()) + 1, 100))

In [ ]:
def visualize_distribution_pie(x):
    
    
    fig, axs = plt.subplots(1, 3, figsize=(15, 5))
    palette = sns.color_palette('pastel')

    # Visualisasi untuk BB/U
    bbu_counts = x.iloc[:,-3].value_counts()
    axs[0].pie(bbu_counts, labels=bbu_counts.index, autopct='%1.1f%%', colors=palette)
    axs[0].set_title('BB/U Distribution')

    # Visualisasi untuk TB/U
    tbu_counts = x.iloc[:,-2].value_counts()
    axs[1].pie(tbu_counts, labels=tbu_counts.index, autopct='%1.1f%%', colors=palette)
    axs[1].set_title('TB/U Distribution')

    # Visualisasi untuk BB/TB
    bbtb_counts = x.iloc[:,-1].value_counts()
    axs[2].pie(bbtb_counts, labels=bbtb_counts.index, autopct='%1.1f%%', colors=palette)
    axs[2].set_title('BB/TB Distribution')
    
    plt.suptitle('Perbandingan Data Pada Tiap Target Klasifikasi', fontsize=16)
    plt.show()


In [ ]:
visualize_distribution_pie(df)

### Keterangan Encoding:
- BB/U -> {'Sangat Kurang': 0, 'Kurang': 1, 'Normal': 2, 'Risiko Lebih': 3}
- TB/U -> {'Sangat Pendek': 0, 'Pendek': 1, 'Normal': 2, 'Tinggi': 3}
- BB/TB -> {'Gizi Buruk': 0, 'Gizi Kurang': 1, 'Gizi Baik': 2, 'Risiko Gizi Lebih': 3, 'Gizi Lebih': 4, 'Obesitas': 5}

## Heatmap

In [ ]:
# Create correlation matrix
corr_matrix = df.corr()

# Create a mask
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

# Set the figure size
plt.figure(figsize=(12, 10))

# Create a heatmap
sns.heatmap(corr_matrix,
            annot=True,
            fmt=".2f",  # Format for the annotations
            mask=mask,  # To hide the upper triangle
            cbar_kws={"shrink": 0.8},  # To adjust the colorbar size
            square=True)  # To make the cells square

plt.show()

# Pemisahan dan Pembagian Data

Pemisahan Fitur dan Target

In [ ]:
fitur = [
        'JK',
         'Usia Saat Ukur',
         'Berat',
         'Tinggi',
        #'LiLA',
        # 'ZS BB/U', 
        # 'ZS TB/U'
        # 'ZS BB/TB'
        ]
# df.columns[:-6]

In [ ]:
X = df[fitur]
y = df[df.columns[-3:]]

Pembagian Data Latih dan Data Uji

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2,
                                                    stratify=y[y.columns[-1]]
                                                    )

In [ ]:
print(X_train.shape, X_test.shape)

In [ ]:
visualize_distribution_pie(y_train)

Global Variabel

In [ ]:
targets = ['BB/U', 'TB/U', 'BB/TB']
metode = ['Decision Tree', 'Naive Bayes', 'Raandom Forest']

# Hyper-Tuning Parameter

## Decision Tree

Penyesuaian Parameter

In [ ]:
# Define the parameter grid to search
param_grid = {
    'estimator__max_depth': [2, 4, 8, 16, 32, 64, None],
    'estimator__min_samples_split': [2, 10, 20, 50, 100, 200, 500]
}

Parameter Test and Evaluation

In [ ]:
# Create a Random Forest classifier
dt_model = DecisionTreeClassifier()

# Multi-output wrapper for Random Forest classifier
multi_output_dt = MultiOutputClassifier(dt_model)

# Grid search with cross-validation
grid_search = GridSearchCV(estimator=multi_output_dt, param_grid=param_grid, cv=5, n_jobs=-1)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Get the best parameters and the best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

In [96]:
df_dt_grid_result = pd.DataFrame(grid_search.cv_results_)

In [99]:

print("Best parameters:", best_params)
print("Best cross-validation score:", best_score)

# Evaluate the best model on the test set
best_model = grid_search.best_estimator_
test_score = best_model.score(X_test, y_test)
print("Test set score of the best model:", test_score)

Best parameters: {'estimator__max_depth': 16, 'estimator__min_samples_split': 2}
Best cross-validation score: 0.6159708648170298
Test set score of the best model: 0.6056338028169014


## Random Forest

Global Variable

Penyesuaian Parameter

In [ ]:
# Define the parameter grid to search
param_grid = {
    'estimator__n_estimators': [10, 20, 60, 100, 200, 400, 600],  # Number of trees in the forest
    'estimator__max_depth': [2, 4, 8, 16, 32, 64, None],      # Maximum depth of the trees
    'estimator__min_samples_split': [2, 10, 20, 50, 100, 200, 500],  # Minimum number of samples required to split a node
}

Parameter Test and Evaluation

In [ ]:
# Create a Random Forest classifier
rf_model = RandomForestClassifier()

# Multi-output wrapper for Random Forest classifier
multi_output_rf = MultiOutputClassifier(rf_model)

# Grid search with cross-validation
grid_search = GridSearchCV(estimator=multi_output_rf, param_grid=param_grid, cv=5, n_jobs=-1)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Get the best parameters and the best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best parameters:", best_params)
print("Best cross-validation score:", best_score)

# Evaluate the best model on the test set
best_model = grid_search.best_estimator_
test_score = best_model.score(X_test, y_test)
print("Test set score of the best model:", test_score)

# Define The Model

## Decision Tree

- Best parameters: {'estimator__max_depth': 30, 'estimator__max_fea'estimator__min_samples_split': 2}
- Best cross-validation score: 0.9620727212610983
- Test set score of the best model: 0.9507042253521126

Fix Parameter

In [ ]:
DT_Model = DecisionTreeClassifier(max_depth=30,
                                  max_features='sqrt',
                                  min_samples_leaf=1,
                                  min_samples_split=2
            )
start_time = time.time()
DT_model = DT_Model.fit(X_train, y_train)
end_time = time.time()

In [ ]:
training_time = end_time - start_time
training_time

In [ ]:
DT_model

## Naive Bayes

In [ ]:
nb_model = GaussianNB() 
NB_Model = MultiOutputClassifier(nb_model, n_jobs=-1)
start_time = time.time()
NB_model = NB_Model.fit(X_train, y_train)
end_time = time.time()

In [ ]:
elapsed_time = end_time - start_time
print("Waktu komputasi:", elapsed_time, "detik")

In [ ]:
NB_model

## Random Forest

- Best parameters: {'estimator__bootstrap': True, 'estimator__max_depth': 10, 'estimator__min_samples_leaf': 2, 'estimator__min_samples_split': 2, 'estimator__n_estimators': 50}
- Best cross-validation score: 0.9977963843588024
- Test set score of the best model: 0.9964788732394366

In [ ]:
RF_Model = RandomForestClassifier(bootstrap= True,
                                  max_depth= 10,
                                  min_samples_leaf=2,
                                  min_samples_split=2,
                                  n_estimators=50)
start_time = time.time()
RF_model = RF_Model.fit(X_train, y_train)
end_time = time.time()

In [ ]:
elapsed_time = end_time - start_time
print("Waktu komputasi:", elapsed_time, "detik")

In [ ]:
RF_model

# Pengujian dan Evaluasi Model

Memprediksi Data test

In [ ]:
DT_ypred = DT_model.predict(X_test).T
RF_ypred = RF_model.predict(X_test).T
NB_ypred = NB_model.predict(X_test).T

Penyeragaman Bentuk Data

In [ ]:
arr_X_test = np.array(X_test.T)

In [ ]:
arr_y_test = np.array(y_test.T)

## Confusion Matrix

In [ ]:
def plot_confusion_matrices(y_prediction, model_name):
    def plot_confusion_matrix(y_prediction, arr_y_test):
        labels = [c for c in range(confusion_matrix(y_prediction, arr_y_test).shape[1])]
        sns.heatmap(confusion_matrix(arr_y_test, y_prediction), annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
        plt.xlabel('True Label')
        plt.ylabel('Predicted Label')
        

    plt.figure(figsize=(24, 5))
    plt.suptitle(f'Confusion Matrix {model_name} Model', fontsize=16)

    for i, y_pred_T in enumerate(y_prediction):
        plt.subplot(1, len(y_prediction), i+1)
        plt.title(f'Confusion Matrix for {targets[i]}')
        plot_confusion_matrix(y_pred_T, arr_y_test[i])
    plt.show()

In [ ]:
model_name = ['Decision Tree', 'Naive Bayes', 'Random Forest']

### Decision Tree

In [ ]:
plot_confusion_matrices(DT_ypred, model_name[0])

### Naive Bayes

In [ ]:
plot_confusion_matrices(NB_ypred, model_name[1])

### Random Forest

In [ ]:
plot_confusion_matrices(RF_ypred, model_name[2])

## Classification Report 

In [ ]:
def report_of_class(y_prediction, model_name):
    print(f"Laporan Klasifikasi Model {model_name}")
    # Inisialisasi list untuk menyimpan laporan klasifikasi
    reports = []

    # Iterasi melalui setiap variabel target
    for i in range(y_test.shape[1]):
        # Hitung laporan klasifikasi untuk variabel target ke-i
        report = classification_report(arr_y_test[i], y_prediction[i])
        # Tambahkan laporan klasifikasi ke dalam list
        reports.append(report)

    # Tampilkan laporan klasifikasi untuk setiap variabel target
    for i, report in enumerate(reports):
        print(f"Laporan Klasifikasi untuk Target {targets[i]}:")
        print(report)
        print("\n")  

### Decision Tree

In [ ]:
report_of_class(DT_ypred, model_name[0])

### Naive Bayes

In [ ]:
report_of_class(NB_ypred, model_name[1])

### Random Forest

In [ ]:
report_of_class(RF_ypred,model_name[2])

## Accuracy

In [ ]:

# Inisialisasi list untuk menyimpan nilai akurasi, presisi, recall, dan F1-score
DT_accuracy = []
NB_accuracy = []
RF_accuracy = []

# Iterasi melalui setiap variabel target
for i in range(len(targets)):
    # Hitung nilai-nilai metrik untuk variabel target ke-i
    DT_accuracy.append(accuracy_score(arr_y_test[i, :], DT_ypred[i,:]))
    NB_accuracy.append(accuracy_score(arr_y_test[i, :], NB_ypred[i,:]))
    RF_accuracy.append(accuracy_score(arr_y_test[i, :], RF_ypred[i,:]))

# Membuat plot
x = np.arange(len(targets))

plt.figure(figsize=(8, 5))
bars1 = plt.bar(x - 0.2, DT_accuracy, 0.2, label='Decision Tree')
bars2 = plt.bar(x, NB_accuracy, 0.2, label='Naive Bayes')
bars3 = plt.bar(x + 0.2, RF_accuracy, 0.2, label='Random Forest')
# bars4 = plt.bar(x + 0.4, f1_scores, 0.2, label='F1-score')

plt.xlabel('Targets')
plt.ylabel('Scores')
plt.title('Accuracy')
plt.xticks(x, targets)
plt.yticks(np.arange(0, 1.1, 0.1))  # Atur label pada sumbu y

# Tambahkan nilai di atas setiap bar
for bars in [bars1, bars2, bars3]:
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width() / 2, height, round(height, 2), ha='center', va='bottom')

plt.legend(loc='lower right')  # Mengatur lokasi legend ke pojok kanan bawah
plt.show()

## Precision

In [ ]:

# Inisialisasi list untuk menyimpan nilai akurasi, presisi, recall, dan F1-score
DT_prec = []
NB_prec = []
RF_prec = []

# Iterasi melalui setiap variabel target
for i in range(len(targets)):
    # Hitung nilai-nilai metrik untuk variabel target ke-i
    DT_prec.append(precision_score(arr_y_test[i, :], DT_ypred[i,:], average='macro'))
    NB_prec.append(precision_score(arr_y_test[i, :], NB_ypred[i,:], average='macro'))
    RF_prec.append(precision_score(arr_y_test[i, :], RF_ypred[i,:], average='macro'))

# Membuat plot
x = np.arange(len(targets))

plt.figure(figsize=(8, 5))
bars1 = plt.bar(x - 0.2, DT_prec, 0.2, label='Decision Tree')
bars2 = plt.bar(x, NB_prec, 0.2, label='Naive Bayes')
bars3 = plt.bar(x + 0.2, RF_prec, 0.2, label='Random Forest')
# bars4 = plt.bar(x + 0.4, f1_scores, 0.2, label='F1-score')

plt.xlabel('Targets')
plt.ylabel('Scores')
plt.title('Precision')
plt.xticks(x, targets)
plt.yticks(np.arange(0, 1.1, 0.1))  # Atur label pada sumbu y

# Tambahkan nilai di atas setiap bar
for bars in [bars1, bars2, bars3]:
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width() / 2, height, round(height, 2), ha='center', va='bottom')

plt.legend(loc='lower right')  # Mengatur lokasi legend ke pojok kanan bawah
plt.show()

## Recall

In [ ]:

# Inisialisasi list untuk menyimpan nilai akurasi, presisi, recall, dan F1-score
DT_rec = []
NB_rec = []
RF_rec = []

# Iterasi melalui setiap variabel target
for i in range(len(targets)):
    # Hitung nilai-nilai metrik untuk variabel target ke-i
    DT_rec.append(recall_score(arr_y_test[i, :], DT_ypred[i,:], average='macro'))
    NB_rec.append(recall_score(arr_y_test[i, :], NB_ypred[i,:], average='macro'))
    RF_rec.append(recall_score(arr_y_test[i, :], RF_ypred[i,:], average='macro'))

# Membuat plot
x = np.arange(len(targets))

plt.figure(figsize=(8, 5))
bars1 = plt.bar(x - 0.2, DT_rec, 0.2, label='Decision Tree')
bars2 = plt.bar(x, NB_rec, 0.2, label='Naive Bayes')
bars3 = plt.bar(x + 0.2, RF_rec, 0.2, label='Random Forest')
# bars4 = plt.bar(x + 0.4, f1_scores, 0.2, label='F1-score')

plt.xlabel('Targets')
plt.ylabel('Scores')
plt.title('Recall')
plt.xticks(x, targets)
plt.yticks(np.arange(0, 1.1, 0.1))  # Atur label pada sumbu y

# Tambahkan nilai di atas setiap bar
for bars in [bars1, bars2, bars3]:
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width() / 2, height, round(height, 2), ha='center', va='bottom')

plt.legend(loc='lower right')  # Mengatur lokasi legend ke pojok kanan bawah
plt.show()

## F1-Score

In [ ]:

# Inisialisasi list untuk menyimpan nilai akurasi, presisi, recall, dan F1-score
DT_f1score = []
NB_f1score = []
RF_f1score = []

# Iterasi melalui setiap variabel target
for i in range(len(targets)):
    # Hitung nilai-nilai metrik untuk variabel target ke-i
    DT_f1score.append(f1_score(arr_y_test[i, :], DT_ypred[i, :], average='macro'))
    NB_f1score.append(f1_score(arr_y_test[i, :], NB_ypred[i, :], average='macro'))
    RF_f1score.append(f1_score(arr_y_test[i, :], RF_ypred[i, :], average='macro'))
    arr_y_test[i, :], RF_ypred[i,:]

# Membuat plot
x = np.arange(len(targets))

plt.figure(figsize=(8, 5))
bars1 = plt.bar(x - 0.2, DT_f1score, 0.2, label='Decision Tree')
bars2 = plt.bar(x, NB_f1score, 0.2, label='Naive Bayes')
bars3 = plt.bar(x + 0.2, RF_f1score, 0.2, label='Random Forest')
# bars4 = plt.bar(x + 0.4, f1_scores, 0.2, label='F1-score')

plt.xlabel('Targets')
plt.ylabel('Scores')
plt.title('F1-Score')
plt.xticks(x, targets)
plt.yticks(np.arange(0, 1.1, 0.1))  # Atur label pada sumbu y

# Tambahkan nilai di atas setiap bar
for bars in [bars1, bars2, bars3]:
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width() / 2, height, round(height, 2), ha='center', va='bottom')

plt.legend(loc='lower right')  # Mengatur lokasi legend ke pojok kanan bawah
plt.show()

## AUC-ROC Score

Define the function

In [ ]:
def compute_yprobs (model, x_test, y_test):
    y_probs = model.predict_proba(x_test)
    target_binarize = []
    n_classes = []

    for i in range (y_test.T.shape[1]):
        y_test_binarize = label_binarize(y_test[i], classes=np.unique(y_test[i]))
        target_binarize.append(y_test_binarize)
        n_classes.append(y_probs[i].shape[1])

    return target_binarize, n_classes, y_probs

In [ ]:
def aucroc_curve(y_binarize, n_classes, y_probs):
    lw = 2
    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_binarize[:, i], y_probs[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(y_binarize.ravel(), y_probs.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
    
    # First aggregate all false positive rates
    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

    # Then interpolate all ROC curves at these points
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(n_classes):
        mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

    # Finally average it and compute AUC
    mean_tpr /= n_classes

    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

    # Plot all ROC curves
    plt.plot(
        fpr["micro"],
        tpr["micro"],
        label="micro-average ROC curve (area = {0:0.2f})".format(roc_auc["micro"]),
        color="deeppink",
        linestyle=":",
        linewidth=4,
    )

    plt.plot(
        fpr["macro"],
        tpr["macro"],
        label="macro-average ROC curve (area = {0:0.2f})".format(roc_auc["macro"]),
        color="navy",
        linestyle=":",
        linewidth=4,
    )

    colors = cycle(["aqua", "darkorange", "cornflowerblue"])
    for i, color in zip(range(n_classes), colors):
        plt.plot(
            fpr[i],
            tpr[i],
            color=color,
            lw=lw,
            label="ROC curve of class {0} (area = {1:0.2f})".format(i, roc_auc[i]),
        )

    plt.plot([0, 1], [0, 1], "k--", lw=lw)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("Receiver operating characteristic (ROC) curves")
    plt.legend(loc="lower right")

Result

### Decision Tree

In [ ]:
target_binarize, n_classes, y_probs = compute_yprobs(DT_model, X_test, arr_y_test)

In [ ]:
plt.figure(figsize=(18, 5))
plt.suptitle('ROC Curves Decision Tree Model', fontsize=16)  # Judul utama

plt.subplot(1, 3, 1)
aucroc_curve(target_binarize[0], n_classes[0], y_probs[0])
plt.title('BB/U')

plt.subplot(1, 3, 2)
aucroc_curve(target_binarize[1], n_classes[1], y_probs[1])
plt.title('TB/U')

plt.subplot(1, 3, 3)
aucroc_curve(target_binarize[2], n_classes[2], y_probs[2])
plt.title('BB/TB')

plt.tight_layout()
plt.show()

### Naive Bayes

In [ ]:
target_binarize, n_classes, y_probs = compute_yprobs(NB_model, X_test, arr_y_test)

In [ ]:
plt.figure(figsize=(18, 5))
plt.suptitle('ROC Curves Naive Bayes Model', fontsize=16)  # Judul utama

plt.subplot(1, 3, 1)
aucroc_curve(target_binarize[0], n_classes[0], y_probs[0])
plt.title('BB/U')

plt.subplot(1, 3, 2)
aucroc_curve(target_binarize[1], n_classes[1], y_probs[1])
plt.title('TB/U')

plt.subplot(1, 3, 3)
aucroc_curve(target_binarize[2], n_classes[2], y_probs[2])
plt.title('BB/TB')

plt.tight_layout()
plt.show()

### Random Forest

In [ ]:
target_binarize, n_classes, y_probs = compute_yprobs(RF_model, X_test, arr_y_test)

In [ ]:
plt.figure(figsize=(18, 5))
plt.suptitle('ROC Curves Random Forest Model', fontsize=16)  # Judul utama

plt.subplot(1, 3, 1)
aucroc_curve(target_binarize[0], n_classes[0], y_probs[0])
plt.title('BB/U')

plt.subplot(1, 3, 2)
aucroc_curve(target_binarize[1], n_classes[1], y_probs[1])
plt.title('TB/U')

plt.subplot(1, 3, 3)
aucroc_curve(target_binarize[2], n_classes[2], y_probs[2])
plt.title('BB/TB')

plt.tight_layout()
plt.show()

# Model Inference

## Menyimpan Model

In [ ]:
pickle.dump(DT_model, open('model_dtm.pkl','wb'))
pickle.dump(NB_model, open('model_nbm.pkl','wb'))
pickle.dump(RF_model, open('model_rfm.pkl','wb'))

## Load Model

### Model Path

In [ ]:
model_path = ['/home/rmfarizky/Project/deteksi_stunting/Deteksi_Stunting/Classification/model_dtm.pkl',
              '/home/rmfarizky/Project/deteksi_stunting/Deteksi_Stunting/Classification/model_nbm.pkl',
              '/home/rmfarizky/Project/deteksi_stunting/Deteksi_Stunting/Classification/model_rfm.pkl']

### Loading Model

In [ ]:
# Load the model
with open(model_path[0], 'rb') as file:
    model_1 = pickle.load(file)

with open(model_path[1], 'rb') as file:
    model_2 = pickle.load(file)
    
with open(model_path[2], 'rb') as file:
    model_3 = pickle.load(file)

In [ ]:
df_shuffled = df.sample(frac=1, random_state=42).reset_index(drop=True)
val = df_shuffled.head()
val[:6]

In [ ]:
val[val.columns[-3:]]

In [ ]:
# Data yang diberikan
x_data = val[fitur]
y_data = val[val.columns[-3:]]
# Membuat DataFrame

In [ ]:
prediction_dtm = model_1.predict(x_data)
prediction_nbm = model_2.predict(x_data)
prediction_rfm = model_3.predict(x_data)

In [ ]:
# Mencetak setiap variabel
print("Prediction DTM:", prediction_dtm)
print("Prediction NBM:", prediction_nbm)
print("Prediction RFM:", prediction_rfm)

# TESTING WITH DATASET

In [ ]:
df1 = pd.read_csv("/home/rmfarizky/Project/deteksi_stunting/DRAFT/Dataset/new_draft/final_data.csv")

In [ ]:
df1.info()

In [ ]:
df1.rename(columns={'usia ukur': 'Usia Saat Ukur'}, inplace=True)

In [ ]:
Xd = df1[fitur]
yd = df1[df.columns[-3:]]

In [ ]:
yd

In [ ]:
predict_dtm = model_1.predict(Xd)
predict_nbm = model_2.predict(Xd)
predict_rfm = model_3.predict(Xd)

In [ ]:
# Inisialisasi list untuk menyimpan laporan klasifikasi
reports = []
targets = ['BB/U', 'TB/U', 'BB/TB']

# Iterasi melalui setiap variabel target
for i in range(y_test.shape[1]):
    # Hitung laporan klasifikasi untuk variabel target ke-i
    report = classification_report(np.array(yd).T[i], predict_rfm.T[i])
    # Tambahkan laporan klasifikasi ke dalam list
    reports.append(report)

# Tampilkan laporan klasifikasi untuk setiap variabel target
for i, report in enumerate(reports):
    print(f"Laporan Klasifikasi untuk Target {targets[i]}:")
    print(report)
    print("\n")
